In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
%reload_ext autoreload
%autoreload 2

from src.constants import csv_file, directory, INPUT_SHAPE, YEAR, ext

from src.utils import (
    get_dataloader,
    sample_iid,
    split_val,
)

from src.model import Model_Retinopathy
from src.server_scaffold import Server_Scaffold
from src.server_fedavg import Server_FedAVG

from src.constants import EPOCHS, BATCH_SIZE, LEARNING_RATE, UPDATES
from src.constants import K_CLIENTS, C, rounds, clients
from src.constants import optimizer_fn

In [4]:
# DATA STUFF
df = pd.read_csv(csv_file)
df, val_df = split_val(df, 0.05)
df = sample_iid(df, 0.3).reset_index(drop=True)
_, val_loader = get_dataloader(val_df, ext, directory, BATCH_SIZE)

In [5]:
# server_fedavg = Server_FedAVG(K_CLIENTS, optimizer_fn, df, val_loader)

In [6]:
# server_fedavg.train_loop(rounds, EPOCHS)

In [7]:
gpu_memory = torch.cuda.memory_allocated()
print(gpu_memory / (1024**2), "MB")
server_scaffold = Server_Scaffold(K_CLIENTS, optimizer_fn, df, val_loader)
""" for key, layer in server_scaffold.model.named_parameters():
    print(key, layer) """
print((torch.cuda.memory_allocated() - gpu_memory) / (1024**2), "MB")

0.0 MB
394.33447265625 MB


In [ ]:
server_scaffold.train_loop(rounds, EPOCHS)